In [6]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import random
import re

def get_soup(url):
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

    driver.get(url)
    time.sleep(1)
    html = driver.page_source

    driver.quit()
    soup = BeautifulSoup(html, 'html.parser')
    return soup


In [7]:
# hrefの値を取得
url = "https://api.coin.z.com/data/trades/BTC/"
soup = get_soup(url)
href_value = soup.find("a", text="2019").get("href")
href_value

C:\Users\yamaguchi\AppData\Local\Temp\ipykernel_20532\2697349742.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  href_value = soup.find("a", text="2019").get("href")


'2019/'

In [8]:
year = url + href_value

'https://api.coin.z.com/data/trades/BTC/2019/'

In [ ]:
get_soup(year)

In [9]:
from datetime import datetime, timedelta

base_url = "https://api.coin.z.com/data/trades/BTC/"
start_date = datetime(2018, 9, 1)
end_date = datetime(2023, 8, 1)

# 月ごとのURLを生成する関数
def generate_monthly_urls(start, end, base):
    current_date = start
    urls = []
    while current_date <= end:
        urls.append(base + current_date.strftime("%Y/%m/"))
        current_date += timedelta(days=31)  # 最大日数を加算して次の月へ移動
        current_date = current_date.replace(day=1)  # 日を1日にリセット
    return urls

monthly_urls = generate_monthly_urls(start_date, end_date, base_url)
monthly_urls

['https://api.coin.z.com/data/trades/BTC/2018/09/',
 'https://api.coin.z.com/data/trades/BTC/2018/10/',
 'https://api.coin.z.com/data/trades/BTC/2018/11/',
 'https://api.coin.z.com/data/trades/BTC/2018/12/',
 'https://api.coin.z.com/data/trades/BTC/2019/01/',
 'https://api.coin.z.com/data/trades/BTC/2019/02/',
 'https://api.coin.z.com/data/trades/BTC/2019/03/',
 'https://api.coin.z.com/data/trades/BTC/2019/04/',
 'https://api.coin.z.com/data/trades/BTC/2019/05/',
 'https://api.coin.z.com/data/trades/BTC/2019/06/',
 'https://api.coin.z.com/data/trades/BTC/2019/07/',
 'https://api.coin.z.com/data/trades/BTC/2019/08/',
 'https://api.coin.z.com/data/trades/BTC/2019/09/',
 'https://api.coin.z.com/data/trades/BTC/2019/10/',
 'https://api.coin.z.com/data/trades/BTC/2019/11/',
 'https://api.coin.z.com/data/trades/BTC/2019/12/',
 'https://api.coin.z.com/data/trades/BTC/2020/01/',
 'https://api.coin.z.com/data/trades/BTC/2020/02/',
 'https://api.coin.z.com/data/trades/BTC/2020/03/',
 'https://ap

In [10]:
start_date_file = datetime(2018, 9, 5)
end_date_file = datetime(2023, 8, 15)

# 日付ごとのファイルURLを生成する関数
def generate_daily_file_urls(start, end, base):
    current_date = start
    urls = []
    while current_date <= end:
        file_name = current_date.strftime("%Y%m%d") + "_BTC.csv.gz"
        urls.append(base + current_date.strftime("%Y/%m/") + file_name)
        current_date += timedelta(days=1)
    return urls

daily_file_urls = generate_daily_file_urls(start_date_file, end_date_file, base_url)
# daily_file_urls[:10]  # 最初の10つのURLを表示してみる
daily_file_urls

['https://api.coin.z.com/data/trades/BTC/2018/09/20180905_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180906_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180907_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180908_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180909_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180910_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180911_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180912_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180913_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180914_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180915_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180916_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180917_BTC.csv.gz',
 'https://api.coin.z.com/data/trades/BTC/2018/09/20180918_BTC.csv.gz',
 'http

In [11]:
import requests
import os

def download_file(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        return None
    return save_path

# ダウンロード先のディレクトリを指定
save_dir = r"H:\マイドライブ\pytest\virtual_currency\gmo\gmo_data\historical_data"  # このパスを適切なものに変更してください

# ダウンロードを開始
downloaded_files = []
for url in daily_file_urls[:10]:
    file_name = os.path.basename(url)
    save_path = os.path.join(save_dir, file_name)
    result = download_file(url, save_path)
    if result:
        downloaded_files.append(result)

In [14]:
import requests
import os
import gzip
from io import BytesIO

def download_and_decompress_file(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        # gzファイルの内容をメモリ上に読み込む
        compressed_file = BytesIO(response.content)
        with gzip.GzipFile(fileobj=compressed_file) as decompressed_file:
            # 解凍された内容を保存する
            with open(save_path, 'wb') as file:
                file.write(decompressed_file.read())
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        return None
    return save_path

# ダウンロード先のディレクトリを指定
save_dir = r"H:\マイドライブ\pytest\virtual_currency\gmo\gmo_data\historical_data"   # このパスを適切なものに変更してください

# ダウンロードと解凍を開始
downloaded_files = []
for url in daily_file_urls:
    file_name = os.path.basename(url).replace('.gz', '')  # gz拡張子を削除
    save_path = os.path.join(save_dir, file_name)
    result = download_and_decompress_file(url, save_path)
    if result:
        downloaded_files.append(result)

Failed to download https://api.coin.z.com/data/trades/BTC/2023/08/20230815_BTC.csv.gz. Status code: 403


: 